In [1]:
import re
import numpy as np
import pandas as pd
import requests,time
import datetime
from scipy import stats
import csv
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import json
import webbrowser
import warnings
import PyPDF2
import io
import requests
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


In [2]:
ls16_sess = [(pd.read_json(f'16session{i}.json', convert_dates=False)) for i in range(2,18)]
ls17_sess = [(pd.read_json(f'17session{i}.json', convert_dates=False)) for i in range(1,13)]

ls_sess = [*ls16_sess ,*ls17_sess]

# * appending all the sessions to the one dataframe
concatdf = pd.concat(ls_sess, ignore_index=True) # ? only 6 sessions rn

# ? cleaning the data for member names and dates 
# *  Date formatting 
concatdf['date'] = concatdf['date'].str.replace('.','')
concatdf['date'] = (concatdf['date'].str[-4:]+ concatdf['date'].str[2:4] + concatdf['date'].str[:2]).apply(int)  # ? date formating
# *  Member name formatting
concatdf['memberstring'] = [','.join(map(str, l)) for l in concatdf['member']] # ? cleaning the data for member
fourth_column = concatdf.pop('memberstring')
concatdf.insert(4,'memberstring',fourth_column)
concatdf["memberstring"].str.strip()

concatdf["type"].str.strip()
# * _________________________________________________________________________



# concatdf.sort_values(by = 'date')


# * total dates, subjects unique ________________________________________________________________
subjects_unique = concatdf['subjects']
subjects_unique.unique()
print(len(subjects_unique.unique()))
print(len(subjects_unique))
session_dates = concatdf['date']
len(session_dates.unique())

#concatdf
# ? Choose a range of dates 
start_date = 20140707
end_date =  20160303
concatdf['questionText'] = concatdf.questionText.str.replace("\n\r\n","")
#concatdf = concatdf.query(f'date >= {start_date}  & date <= {end_date}') # 20140707 to 20230811
warnings.filterwarnings("ignore",'This pattern is interpreted as a regular expression, and has match groups.To actually get the groups, use str.extract.')
concatdf = concatdf.sort_values(by = 'date', ignore_index= True)

loks16_mem = pd.read_excel("LS16MEMSTATE.xlsx")
loks17_mem = pd.read_excel("LS17MEMSTATE.xlsx")

108948
134866


In [66]:

def ls_frame(start_date, end_date):
   """_returns the datframe with question for the concerned time frame_

    Args:
        start_date (_int_): _the starting date >= 20140707 for now_
        end_date (_type_): _the last date <= 20230811_

    Returns:
        _dataframe_: _with date constraints_
    """   ''''''
   ls_frame = concatdf.query(f'date >= {start_date}  & date <= {end_date}') # 20140707 to 20230811
   return ls_frame


def pdf_indexes(dataframe):
    '''returns indexes of dataframe for column questionsFilePath, links which are pdf files'''
    
    alltr_indexes = []
    allfa_indexes = []

    for i in range(19097,dataframe.index[-1]):          #for word cloud
        val = dataframe.questionsFilePath[i]
        if 'pdf' in val:
            alltr_indexes.append(i)
        else:
            allfa_indexes.append(i)

    return alltr_indexes, allfa_indexes




def subjects_uniqueliststr(dataframe):
    '''Makes all the subjects as string
        for a dataframe, returns a string of all the subjects'''
    subjects_unique = dataframe['ministry']
    subjects_unique_list =  list(subjects_unique)
    subjects_unique_liststr = str(subjects_unique_list)   #for word cloud
    subjects_unique_liststr
    return subjects_unique_liststr




def scrape_pdf_and_questions(dataframe):
  '''Can scrape questions from pdf links in rows after regex confi and paste it on qtext columns
  given a dataframe'''

   #if "pdf" in url:
  indices = pdf_indexes(dataframe)[0]
  for ind in indices:                               # ? could just making a list of strings and pasting them sideways be faster
    url = dataframe.questionsFilePath[ind]
    if "https://" in url:
      r = requests.get(url)
      f = io.BytesIO(r.content)
      reader = PyPDF2.PdfReader(f)
      contents = reader.pages[0].extract_text().split('\n')
      string_contents = str(contents)
      dataframe.loc[ind,'questionText'] = string_contents
      #have to add regular_expressions
    
      #listn = [','.join(map(str, l)) for l in contents]



def total_club_allmem(dataframe):
    
    names = dataframe.loc[:,["member","memberstring"]]#.reset_index()
    #names = names.drop(["index"], axis = 1)
    clubbed_questions_df = names[names.loc[:,"member"].apply(len) != 1 ]
    return clubbed_questions_df


def clubbed_questions_of_a_member(member_name,dataframe):
    member = dataframe.loc[dataframe["memberstring"].str.contains(member_name, case=False)]
    names = member.loc[:,:].reset_index()
    names = names.drop(["index"], axis = 1)
    clubbed_questions_df = names[names.loc[:,"memberstring"] != member_name]
    return clubbed_questions_df



def  members_who_asked_quest(dataframe):
   ''' returns a list of members who asked questions either independently
   or clubbed, and total number of members who asked questions'''
   lok_sabha_members = [dataframe.loc[i,'member'][0] for i in dataframe.index]
   for i in total_club_allmem(dataframe).index:
    for j in range(len(dataframe.loc[i,'member'])):
        lok_sabha_members.append(dataframe.loc[i,'member'][j])
    
    lok_sabha_members = (set(lok_sabha_members))
    length = len(lok_sabha_members)
    return lok_sabha_members, length
   


def get_star_unstar_tot_club(member_name,dataframe):
    
    member_name = member_name
    #warnings.filterwarnings("ignore", 'This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.')
    the_name = dataframe.loc[dataframe['memberstring'].str.contains(member_name,case = False)]
    clubbed_list = []

    club_count = clubbed_questions_of_a_member(member_name,dataframe).shape[0]


    unstarred = 0 if len(the_name.type.value_counts()) < 1 else the_name.type.value_counts().iloc[0]
    starred   = 0 if len(the_name.type.value_counts()) < 2 else the_name.type.value_counts().iloc[1] 
    dictionary_name_ques = {'MemberName' :f'{member_name}',
                            'Total_Questions' : unstarred+starred,
                            'Clubbed': club_count,
                            'Unstarred': unstarred,
                            'Starred' : starred,
                            '% Starred' : 1+starred/(1+unstarred+starred) -1

    }
    return dictionary_name_ques

list = ['a','b']
def comparison_df(dataframe,lsno_member_list_states):
   '''retuens a dataframe for a given dataframe with members total_questions asked prop of 
   clubbed, starred and unstarred'''
   data = []
   for member_name in members_who_asked_quest(dataframe)[0]:
    data.append(get_star_unstar_tot_club(member_name,dataframe))
    comparison_df = pd.DataFrame.from_dict(data)
    comparison_df.MemberName = comparison_df.MemberName.replace(r'\s+', ' ', regex=True)
   return compdf_with_states(comparison_df.sort_values(by = "Starred", ascending = False, ignore_index= True),lsno_member_list_states)

def ministry_frequency(dataframe):
    '''returns a dataframe for a given dataframe highlighting, ministries and the amount of questions asked to them with
    respective proportion'''
    ministry_unique = dataframe['ministry']
    ministry_unique = pd.DataFrame(dataframe['ministry'].value_counts())
    ministry_unique['proportion%'] = ((dataframe['ministry'].value_counts())/dataframe['ministry'].shape[0])*100
    ministry_unique.loc['Column_Total']= ministry_unique.sum(numeric_only=True, axis=0)
    ministry_unique.reset_index(inplace=True)
    return ministry_unique


def compdf_with_states(dataframe,lsno_member_list_states):
    '''comparison df goes into this, with cleaned member_list(second parameter) of states and party'''
    dataframe["State"] = "N/A"
    dataframe["Party Name"] = "N/A"
    for i in range(dataframe.shape[0]):
        ser = lsno_member_list_states.loc[:,"Name of Member"].str.contains(dataframe.loc[i,"MemberName"],case= False)
        
        if len(ser[ser].index) != 0:
            dataframe.loc[i,"State"] = str(lsno_member_list_states.loc[ser[ser].index[0],"State"])
            dataframe.loc[i,"Party Name"] = str(lsno_member_list_states.loc[ser[ser].index[0],"Party Name"])
        
    return dataframe



def clubbed_members_freq(member,dataframe):

    membersask = members_who_asked_quest(dataframe)[0]
    member = clubbed_questions_of_a_member(member,dataframe)
    #supriya = supriya[supriya.memberstring.str.contains("DHANANJAY BHIMRAO MAHADIK", case = False)].shape[0]
   
    club_members = []
    for membs in sorted(membersask):
        count = member[member.memberstring.str.contains(membs, case = False)].shape[0]
        club_mem = {f"MemberName": membs,
                    "Questions_clubbed": count           
        }
        club_members.append(club_mem)
    club_members   = pd.DataFrame.from_dict(club_members)
    return club_members.sort_values(by = "Questions_clubbed", ascending = False)[:].reset_index()

def question_subject2mem(mem_list,dataframe):
    """_summary_

    Args:
        mem_list (_type_): _description_
        dataframe (_type_): _description_

    Returns:
        _type_: _description_
    """     
    mem = dataframe.loc[dataframe['memberstring'].str.contains(mem_list[0], case= False)]
    for i in range(1,len(mem_list)):
        mem = mem.loc[mem['memberstring'].str.contains(mem_list[i], case = False)].reset_index().drop("index", axis=1)
    return mem


















What a member of a certain party clubs with another member of another party
    
1. So what type of questions, ministries  Supriya sule clubs with Eknath shinde?
2. Cleaning the warnings
3. Data plotting

In [4]:
ls_20142016 = ls_frame(20140707,20160303)

In [29]:
warnings.filterwarnings("ignore", 'This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.')
top_sttared = comparison_df(ls_20142016,loks16_mem)

In [73]:
sss = clubbed_members_freq("JAYAKUMAR JAYAVARDHAN",ls_20142016)


#members_state_unident = top_sttared[top_sttared.loc[:,"State"] == "N/A"] # primarily due to either loks_16 not having the data 
#top_sttared.loc[:,["State"]] == "Maharashtra"

compdf_with_states(sss,loks16_mem)[:50]

list_2mem = ["JAYAKUMAR JAYAVARDHAN","SUPRIYA SADANAND SULE"]

#chk = question_subject2mem(list_2mem,ls_20142016)
top_sttared = comparison_df(ls_20142016,loks16_mem)


C:\Users\shubh\AppData\Local\Temp\ipykernel_5756\1501088395.py:164: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  count = member[member.memberstring.str.contains(membs, case = False)].shape[0]
C:\Users\shubh\AppData\Local\Temp\ipykernel_5756\1501088395.py:146: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ser = lsno_member_list_states.loc[:,"Name of Member"].str.contains(dataframe.loc[i,"MemberName"],case= False)
C:\Users\shubh\AppData\Local\Temp\ipykernel_5756\1501088395.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  the_name = dataframe.loc[dataframe['memberstring'].str.contains(member_name,case = False)]
C:\Users\shubh\AppData\Local\Temp\ipykernel_5756\1501088395.py:75: UserWarning: This pattern is interpreted as 

In [150]:
i = "DHANANJAY BHIMRAO MAHADIK"
supriya = clubbed_questions_of_a_member("SUPRIYA SADANAND SULE",ls_20142016)
count = supriya[supriya.memberstring.str.contains(i, case = False)].shape[0]
count

357

In [84]:
top_starred20 = top_sttared[:20]
top_starred = compdf_with_states(top_starred,loks16_mem)
top_starred.reset_index().drop("index", axis=1)
(top_starred20.loc[:,["State"]] == "Maharashtra").value_counts()


C:\Users\shubh\AppData\Local\Temp\ipykernel_5756\1501088395.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["State"] = "N/A"
C:\Users\shubh\AppData\Local\Temp\ipykernel_5756\1501088395.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Party Name"] = "N/A"


State
False    11
True      9
Name: count, dtype: int64

In [ ]:
ministry_frequency(ls_20142016)